# Coursera Assignment

### First we import the needed libraries

In [87]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import folium
from sklearn.cluster import KMeans
import requests
import json
import pandas as pd
import matplotlib as plt
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

In [88]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')
table = soup.find('tbody')

RowList = []

for rows in table.find_all('tr'):
    row = rows.text.replace('\n', ', \n')
    RowList.append(row.split('\n'))
RowList

[[', ', 'Postcode, ', 'Borough, ', 'Neighbourhood, ', ''],
 [', ', 'M1A, ', 'Not assigned, ', 'Not assigned, ', ''],
 [', ', 'M2A, ', 'Not assigned, ', 'Not assigned, ', ''],
 [', ', 'M3A, ', 'North York, ', 'Parkwoods, ', ''],
 [', ', 'M4A, ', 'North York, ', 'Victoria Village, ', ''],
 [', ', 'M5A, ', 'Downtown Toronto, ', 'Harbourfront, ', ''],
 [', ', 'M5A, ', 'Downtown Toronto, ', 'Regent Park, ', ''],
 [', ', 'M6A, ', 'North York, ', 'Lawrence Heights, ', ''],
 [', ', 'M6A, ', 'North York, ', 'Lawrence Manor, ', ''],
 [', ', 'M7A, ', "Queen's Park, ", 'Not assigned, ', ''],
 [', ', 'M8A, ', 'Not assigned, ', 'Not assigned, ', ''],
 [', ', 'M9A, ', 'Etobicoke, ', 'Islington Avenue, ', ''],
 [', ', 'M1B, ', 'Scarborough, ', 'Rouge, ', ''],
 [', ', 'M1B, ', 'Scarborough, ', 'Malvern, ', ''],
 [', ', 'M2B, ', 'Not assigned, ', 'Not assigned, ', ''],
 [', ', 'M3B, ', 'North York, ', 'Don Mills North, ', ''],
 [', ', 'M4B, ', 'East York, ', 'Woodbine Gardens, ', ''],
 [', ', 'M4B, ', '

In [89]:
#read it into a DataFrame and drop empty columns
df = pd.DataFrame(RowList, columns=['empty', 'PostalCode','Borough','Neighborhood','empty2'])
df.drop(columns=['empty','empty2'], inplace=True)
df.drop(0, inplace=True)

#drop all rows that do not have a Borough assigned
df.drop(df[df.Borough=='Not assigned, '].index, axis=0, inplace=True)
df.head(3)

,PostalCode,Borough,Neighborhood
3,"M3A,","North York,","Parkwoods,"
4,"M4A,","North York,","Victoria Village,"
5,"M5A,","Downtown Toronto,","Harbourfront,"


In [90]:
# If Neighborhood is not assigned, then use assigned Borough value
df.loc[df['Neighborhood'] == 'Not assigned, ', 'Neighborhood'] = df['Borough']

In [91]:
# Group by postal code and Borough and add the neighborhoods
newdf = df.groupby(['PostalCode','Borough']).sum()
newdf=newdf.reset_index()

newdf.head(2)

,PostalCode,Borough,Neighborhood
0,"M1B,","Scarborough,","Rouge, Malvern,"
1,"M1C,","Scarborough,","Highland Creek, Rouge Hill, Port Union,"


In [92]:
# format data (drop the commas)
newdf.PostalCode = newdf.PostalCode.apply(lambda x:x[:-2])
newdf.Borough = newdf.Borough.apply(lambda x: x[:-2])
newdf.Neighborhood = newdf.Neighborhood.apply(lambda x: x[:-2])

newdf.head(2)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"


## Answer to Question #1

In [93]:
newdf.head(10)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [94]:
newdf.shape

(103, 3)

## Onto the next question...

In [95]:
#read the provided csv file
lldf = pd.read_csv('http://cocl.us/Geospatial_data')
lldf.head(2)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497


In [96]:
# perform an inner join on the two data frames to get latitude and longitude into the dataframe
finaldf = pd.merge(newdf, lldf, how='inner',left_on = 'PostalCode', right_on = 'Postal Code')
finaldf.head(2)

,PostalCode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",M1C,43.784535,-79.160497


In [97]:
#drop the duplicate Postal Code column
finaldf.drop(['Postal Code'],axis=1, inplace=True)
finaldf.head(2)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497


## Answer to Question #2

In [98]:
finaldf.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [99]:
finaldf.shape

(103, 5)

## Onto the next and final question...

In [100]:
# Initialized longitude and latitude with Toronto coordinates and creating folium circlemarkers
Latitude = 43.6532
Longitude = -79.3832
map_toronto = folium.Map(location =[Latitude,Longitude],zoom_start=11)

for PostalCode,Borough,Neighborhood,Latitude,Longitude in zip(finaldf['PostalCode'],finaldf['Borough'],finaldf['Neighborhood'],finaldf['Latitude'],finaldf['Longitude']):
    label = PostalCode
    folium.CircleMarker(
        [Latitude,Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)
map_toronto

### Assigning Foursquare credentials to variables CLIENT_ID and CLIENT_SECRET - Cell hidden for security purposes

In [101]:
# The code was removed by Watson Studio for sharing.

Credentials assigned successfully!


In [102]:
#This is the function to retrive venues for a given longitude and latitude taken from the lab
limit = 100
radius = 500
def get_nearby_venues(postcode,borough, name,latitude,longitude,radius=500):
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        latitude,
        longitude,
        radius,
        limit)
    results = requests.get(url).json()
    
    list =[]
    new_results= results['response']['groups']
    for level1 in new_results:
        for level2 in level1['items']:
            venue_name = level2['venue']['name']
            venue_longitude=level2['venue']['location']['lng']
            venue_latitude = level2['venue']['location']['lat']
            for level3 in level2['venue']['categories']:
                venue_category = level3['shortName']
            list.append([postcode,borough,name,latitude,longitude,venue_category,venue_name,venue_longitude,venue_latitude])
        
        
        df = pd.DataFrame(list,columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude','Category','Venue_Name','Venue_Longitude','Venue_Latitude'])
        return df

In [103]:
#initialize an empty data frame
venuedf_all = pd.DataFrame(columns=['PostalCode','Borough','Neighborhood','Latitude','Longitude','Category','Venue_Name','Venue_Longitude','Venue_Latitude'])

#iterate through all the entries and add the venues data frame returned by get_nearby_venues to the main DataFrame venuedf_all
for postcode,borough,neighborhood,latitude,longitude in zip(finaldf['PostalCode'],finaldf['Borough'],finaldf['Neighborhood'],finaldf['Latitude'],finaldf['Longitude']):
    venuedf = get_nearby_venues(postcode,borough,neighborhood,latitude,longitude)
    if isinstance(venuedf, pd.DataFrame):
        venuedf_all = pd.concat([venuedf_all,venuedf])
    else:
        print ("No venues : " & postcode,borough,neighborhood,latitude,longitude)

In [104]:
venuedf_all.head(5)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Category,Venue_Name,Venue_Longitude,Venue_Latitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,Fast Food,Wendy's,-79.199056,43.807448
0,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Bar,Royal Canadian Legion,-79.163085,43.782533
0,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Pizza,Swiss Chalet Rotisserie & Grill,-79.189914,43.767697
1,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Electronics,G & G Electronics,-79.191537,43.765309
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,Mexican,Big Bite Burrito,-79.190720,43.766299


In [105]:
# one hot encoding
venuedf_all_onehot = pd.get_dummies(venuedf_all['Category'],prefix="",prefix_sep="")

#add Neighborhood column to the dataframe
venuedf_all_onehot.insert(loc=0,column='Neighborhood_Name',value=venuedf_all['Neighborhood'])
venues_grouped = venuedf_all_onehot.groupby('Neighborhood_Name').sum().reset_index()
venues_grouped.head(5)

,Neighborhood_Name,Accessories,Adult Boutique,Afghan,Airport,Airport Service,American,Antiques,Apparel,Aquarium,...,Vegetarian / Vegan,Video Games,Video Store,Vietnamese,Warehouse Store,Wine Bar,Wings,Women's Store,Yoga Studio,Yogurt
0,"Adelaide, King, Richmond",0,0,0,0,0,4,0,3,0,...,1,0,0,0,0,1,0,1,0,0
1,Agincourt,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### .....

In [106]:
def return_common_venues (neighborhood, no_of_venues):
    temp = venues_grouped[venues_grouped.Neighborhood_Name == neighborhood].T.reset_index()
    
    categories = temp.iloc[1:]
    sorted_categories = categories.sort_values(by=categories.columns.values[1],  ascending= False)
    topx = sorted_categories.head(no_of_venues).drop(categories.columns.values[1],axis=1).T.reset_index(drop=True)
    topx.insert(loc=0, column='Neighborhood', value = neighborhood)
    topxlist = topx.values.tolist()
    
    # this is for a second data frame that also stores # of venues
    temp.columns = ['venue','frequency']
    temp = temp[temp.venue != 'Neighborhood_Name']
    temp.sort_values('frequency', ascending = False, inplace = True)
    temp.reset_index(drop=True, inplace = True)
    temp.insert(loc=0,column='venue_rank', value = 1+ temp.index.values)
    temp.insert(loc=0,column='Neighborhood', value = neighborhood)
    
    return temp.head(no_of_venues), topxlist

In [107]:
no_of_venues=10
first_time = True
#created an empty list and an empty data frame to collect the information returned by the function above
venues_sorted_list=[]
venuesrankeddf_all = pd.DataFrame(columns = ['neighborhood','venue_rank','venue','frequency'])
for neighborhood in venues_grouped.Neighborhood_Name:
    venuesrankeddf, toplist = return_common_venues(neighborhood, no_of_venues)
    venues_sorted_list.extend(toplist)
    if isinstance(venuesrankeddf, pd.DataFrame):
        if first_time:
            venuesrankeddf_all = venuesrankeddf
            first_time = False
        else:
            venuesrankeddf_all = pd.concat([venuesrankeddf_all,venuesrankeddf])            
venues_sorted_all = pd.DataFrame(venues_sorted_list,columns = ['Neighborhood','1','2','3','4','5','6','7','8','9','10'])

In [108]:
#for later clustering
venues_sorted_all.head(3)

,Neighborhood,1,2,3,4,5,6,7,8,9,10
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,American,Thai,Gym,Restaurant,Bakery,Asian,Bar
1,Agincourt,Lounge,Breakfast,Skating Rink,Apparel,Accessories,Neighborhood,Music Venue,Music Store,Museum,Moving Target
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Playground,Park,Accessories,Motel,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Molecular Gastronomy


In [109]:
venuesrankeddf_all.sort_values(by = ['frequency','Neighborhood'],ascending = False)

,Neighborhood,venue_rank,venue,frequency
0,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,14
0,"Design Exchange, Toronto Dominion Centre",1,Coffee Shop,14
0,Central Bay Street,1,Coffee Shop,14
0,"The Danforth West, Riverdale",1,Greek,10
0,Queen's Park,1,Coffee Shop,10
0,"Fairview, Henry Farm, Oriole",1,Apparel,10
0,"Commerce Court, Victoria Hotel",1,Coffee Shop,10
0,Stn A PO Boxes 25 The Esplanade,1,Coffee Shop,9
0,"Ryerson, Garden District",1,Coffee Shop,9
1,"Ryerson, Garden District",2,Apparel,9


In [110]:
# number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood_Name', 1)

# k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# print cluster labels 
kmeans.labels_ 


array([4, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 3, 2, 0,
       2, 2, 2, 2, 4, 0, 2, 2, 2, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 4, 2, 2,
       2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 2, 2, 0, 2, 2, 2, 0, 1, 2, 2, 1, 1, 0, 0, 2, 2, 0, 2, 2, 2, 2,
       2, 2, 0, 2, 2, 2, 2, 2], dtype=int32)

In [111]:
# add clusters to the DataFrame
venues_grouped['cluster'] = kmeans.labels_ 
venues_grouped.head(5)

,Neighborhood_Name,Accessories,Adult Boutique,Afghan,Airport,Airport Service,American,Antiques,Apparel,Aquarium,...,Video Games,Video Store,Vietnamese,Warehouse Store,Wine Bar,Wings,Women's Store,Yoga Studio,Yogurt,cluster
0,"Adelaide, King, Richmond",0,0,0,0,0,4,0,3,0,...,0,0,0,0,1,0,1,0,0,4
1,Agincourt,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
2,"Agincourt North, L'Amoreaux East, Milliken, St...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,2
4,"Alderwood, Long Branch",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [112]:
venues_grouped.shape[0]

100

In [113]:
#add latitude, longitude and Borough columns to the DataFrame
venues_grouped.insert(loc=1,column='longitude',value=0.0)
venues_grouped.insert(loc=2,column='latitude',value=0.0)
venues_grouped.insert(loc=3,column='borough',value="")

In [114]:
#assigning values from finaldf
venues_grouped.loc[venues_grouped.Neighborhood_Name.isin(finaldf.Neighborhood),['longitude']] = finaldf['Longitude']
venues_grouped.loc[venues_grouped.Neighborhood_Name.isin(finaldf.Neighborhood),['latitude']] = finaldf['Latitude']
venues_grouped.loc[venues_grouped.Neighborhood_Name.isin(finaldf.Neighborhood),['borough']] = finaldf['Borough']

In [115]:
#add cluster column to the sorted venues DataFrame sfor cluster analysis 
venues_sorted_all.insert(loc=1,column='cluster', value = 5)

In [116]:
venues_sorted_all.loc[venues_sorted_all.Neighborhood.isin(venues_grouped.Neighborhood_Name),['cluster']] = venues_grouped['cluster']

In [117]:
venues_sorted_all.head(5)

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
0,"Adelaide, King, Richmond",4,Coffee Shop,Café,Steakhouse,American,Thai,Gym,Restaurant,Bakery,Asian,Bar
1,Agincourt,2,Lounge,Breakfast,Skating Rink,Apparel,Accessories,Neighborhood,Music Venue,Music Store,Museum,Moving Target
2,"Agincourt North, L'Amoreaux East, Milliken, St...",2,Playground,Park,Accessories,Motel,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Molecular Gastronomy
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",2,Grocery Store,Beer Store,Sandwiches,Pizza,Pharmacy,Fried Chicken,Video Store,Coffee Shop,Fast Food,Movie Theater
4,"Alderwood, Long Branch",2,Pizza,Sandwiches,Pub,Pharmacy,Coffee Shop,Skating Rink,Gym,Moving Target,Neighborhood,Music Venue


In [118]:
#create folium map
map_clusters = folium.Map(location=[latitude,longitude], zoom_start =11)

#set color scheme for the clusters
x = np.arange(kclusters+1)
ys = [i+x+(i*x)**2 for i in range(kclusters+1)]
print(ys)
colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
marker_colors = []

[array([0, 1, 2, 3, 4, 5]), array([ 1,  3,  7, 13, 21, 31]), array([  2,   7,  20,  41,  70, 107]), array([  3,  13,  41,  87, 151, 233]), array([  4,  21,  70, 151, 264, 409]), array([  5,  31, 107, 233, 409, 635])]


In [119]:
#plotting the K mean cluster on the map
for lat, lon, poi, cluster in zip(venues_grouped['latitude'], venues_grouped['longitude'], venues_grouped['Neighborhood_Name'], venues_grouped['cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [120]:
venues_sorted_all.loc[venues_sorted_all.cluster == 0, venues_sorted_all.columns[list(range(0, venues_sorted_all.shape[1]))]]

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
8,Berczy Park,0,Coffee Shop,Cocktail,Restaurant,Italian,Bakery,Beer Bar,Steakhouse,Farmer's Market,Seafood,Pub
15,"Cabbagetown, St. James Town",0,Coffee Shop,Restaurant,Café,Pizza,Bakery,Pub,Park,Italian,Beer Store,Market
22,Church and Wellesley,0,Japanese,Sushi,Coffee Shop,Gay Bar,Restaurant,Burgers,Fast Food,Bubble Tea,Mediterranean,Men's Store
28,Davisville,0,Desserts,Pizza,Sandwiches,Italian,Thai,Sushi,Café,Seafood,Coffee Shop,Farmer's Market
42,"Fairview, Henry Farm, Oriole",0,Apparel,Fast Food,Coffee Shop,Toys & Games,Bakery,Women's Store,Asian,Food Court,Salon / Barbershop,Sandwiches
48,"Harbord, University of Toronto",0,Café,Coffee Shop,Bakery,Gym,Bookstore,Japanese,Restaurant,Bar,Beer Bar,Nightclub
50,"Harbourfront, Regent Park",0,Coffee Shop,Café,Park,Bakery,Theater,Mexican,Breakfast,Restaurant,Pub,Farmer's Market
64,"Little Portugal, Trinity",0,Bar,Men's Store,Restaurant,Asian,Coffee Shop,Café,Pizza,Cocktail,Bakery,Vietnamese
73,Queen's Park,0,Coffee Shop,Japanese,Sushi,Gym,Diner,Burgers,Café,Bubble Tea,Mexican,Music Venue
77,"Runnymede, Swansea",0,Coffee Shop,Pizza,Café,Italian,Sushi,Sandwiches,Gym,Bar,Gourmet,Supplement Shop


## Cluster 2

In [121]:
venues_sorted_all.loc[venues_sorted_all.cluster == 1, venues_sorted_all.columns[list(range(0, venues_sorted_all.shape[1]))]]

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
19,Central Bay Street,1,Coffee Shop,Café,Italian,Burgers,Bar,Salad,Spa,Chinese,Sandwiches,Thai
49,"Harbourfront East, Toronto Islands, Union Station",1,Coffee Shop,Hotel,Aquarium,Pizza,Café,Brewery,Italian,Bakery,Restaurant,Scenic Lookout
78,"Ryerson, Garden District",1,Coffee Shop,Apparel,Café,Cosmetics,Middle Eastern,Theater,Restaurant,Diner,Ramen,Plaza
81,St. James Town,1,Coffee Shop,Restaurant,Café,Hotel,Apparel,Bakery,Park,Italian,Cocktail,Breakfast
82,Stn A PO Boxes 25 The Esplanade,1,Coffee Shop,Restaurant,Café,Seafood,Beer Bar,Pub,Cocktail,Hotel,Italian,Creperie


## Cluster 3

In [122]:
venues_sorted_all.loc[venues_sorted_all.cluster == 2, venues_sorted_all.columns[list(range(0, venues_sorted_all.shape[1]))]]

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
1,Agincourt,2,Lounge,Breakfast,Skating Rink,Apparel,Accessories,Neighborhood,Music Venue,Music Store,Museum,Moving Target
2,"Agincourt North, L'Amoreaux East, Milliken, St...",2,Playground,Park,Accessories,Motel,Music Venue,Music Store,Museum,Moving Target,Movie Theater,Molecular Gastronomy
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",2,Grocery Store,Beer Store,Sandwiches,Pizza,Pharmacy,Fried Chicken,Video Store,Coffee Shop,Fast Food,Movie Theater
4,"Alderwood, Long Branch",2,Pizza,Sandwiches,Pub,Pharmacy,Coffee Shop,Skating Rink,Gym,Moving Target,Neighborhood,Music Venue
5,"Bathurst Manor, Downsview North, Wilson Heights",2,Coffee Shop,Yogurt,Fried Chicken,Mall,Sushi,Fast Food,Bank,Sandwiches,Grocery Store,Restaurant
6,Bayview Village,2,Café,Bank,Chinese,Japanese,Accessories,Nightclub,New American,Neighborhood,Music Venue,Music Store
7,"Bedford Park, Lawrence Manor East",2,Fast Food,Italian,Coffee Shop,Indian,Restaurant,Breakfast,Butcher,Café,Sandwiches,Sushi
9,"Birch Cliff, Cliffside West",2,Stadium,Entertainment,Skating Rink,Café,Accessories,Moving Target,Music Venue,Music Store,Museum,Motel
10,"Bloordale Gardens, Eringate, Markland Wood, Ol...",2,Café,Convenience Store,Beer Store,Pizza,Liquor Store,Pharmacy,Museum,New American,Neighborhood,Music Venue
11,"Brockton, Exhibition Place, Parkdale Village",2,Breakfast,Café,Coffee Shop,Pet Store,Caribbean,Grocery Store,Gym,Climbing Gym,Furniture / Home,Performing Arts


## Cluster 4

In [123]:
venues_sorted_all.loc[venues_sorted_all.cluster == 3, venues_sorted_all.columns[list(range(0, venues_sorted_all.shape[1]))]]

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
20,"Chinatown, Grange Park, Kensington Market",3,Café,Bar,Vietnamese,Vegetarian / Vegan,Coffee Shop,Dumplings,Bakery,Mexican,Chinese,Dim Sum


## Cluster 5

In [125]:
venues_sorted_all.loc[venues_sorted_all.cluster == 4, venues_sorted_all.columns[list(range(0, venues_sorted_all.shape[1]))]]

,Neighborhood,cluster,1,2,3,4,5,6,7,8,9,10
0,"Adelaide, King, Richmond",4,Coffee Shop,Café,Steakhouse,American,Thai,Gym,Restaurant,Bakery,Asian,Bar
27,"Commerce Court, Victoria Hotel",4,Coffee Shop,Café,Hotel,Restaurant,American,Deli / Bodega,Gastropub,Seafood,Bakery,Gym
32,"Design Exchange, Toronto Dominion Centre",4,Coffee Shop,Hotel,Café,American,Restaurant,Italian,Gastropub,Deli / Bodega,Gym,Seafood
43,"First Canadian Place, Underground city",4,Café,Coffee Shop,Hotel,Restaurant,American,Gym,Gastropub,Bar,Bakery,Seafood
